# * Import : "CDSAPPO.DIM_MOOC_AREA"
    All data

## Parameter

In [4]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2026, 1, 29)

## ETL Process...

### Step 1 : DIM_MOOC_AREA
    DB -> CSV file

In [5]:
''' File Config '''

op_dir = 'data'
op_file = 'dim_mooc_area.csv'

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

with open('SQL/Import-DIM_MOOC_AREA.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()

# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()


try:
    print(f'\nProcessing...')

    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> src_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_file}', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_file}" successfully')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2026-01-29, 14:41:38

TDMDBPR : Connected

Processing...

   -> src_df : 9274 rows, 42 columns

   -> Generate "dim_mooc_area.csv" successfully

TDMDBPR : Disconnected

Job Done !!!


In [6]:
src_df.tail()

,ZONE_TYPE,TEAM_CODE,SCAB_CODE,CCAATT,PROVINCE_KEY,PROVINCE_TH,DISTRICT_TH,SUB_DISTRICT_TH,PROVINCE_ENG,DISTRICT_EN,...,REMARK,SPECIAL_AA,ORG_GRAD,D_CLUSTER,D_HOP_HINT,HOP_HINT_TH,HOP_HINT_EN,TDS_GMD,ORGID_HH,ORG_LEVEL_HH
9269,UPC,U5688,730101000000,730101,นครปฐม_เมืองนครปฐม_พระปฐมเจดีย์,นครปฐม,เมืองนครปฐม,พระปฐมเจดีย์,Nakhon Pathom,Mueang Nakhon Pathom,...,Not Change,Normal,[A],NAKHON PATHOM,NAKHON PATHOM,Nakhon Pathom,NAKHON PATHOM,Central-West,902037,HH
9270,UPC,U5776,730602000000,730602,นครปฐม_สามพราน_ทรงคนอง,นครปฐม,สามพราน,ทรงคนอง,Nakhon Pathom,Sam Phran,...,Not Change,Normal,[A],NAKHON PATHOM,NAKHON PATHOM,Nakhon Pathom,NAKHON PATHOM,Central-West,902037,HH
9271,UPC,U5778,730604000000,730604,นครปฐม_สามพราน_บางกระทึก,นครปฐม,สามพราน,บางกระทึก,Nakhon Pathom,Sam Phran,...,Not Change,Normal,[A],NAKHON PATHOM,NAKHON PATHOM,Nakhon Pathom,NAKHON PATHOM,Central-West,902037,HH
9272,UPC,U5790,730616000000,730616,นครปฐม_สามพราน_อ้อมใหญ่,นครปฐม,สามพราน,อ้อมใหญ่,Nakhon Pathom,Sam Phran,...,Not Change,Normal,[A],NAKHON PATHOM,NAKHON PATHOM,Nakhon Pathom,NAKHON PATHOM,Central-West,902037,HH
9273,UPC,U5835,750102000000,750102,สมุทรสงคราม_เมืองสมุทรสงคราม_บางขันแตก,สมุทรสงคราม,เมืองสมุทรสงคราม,บางขันแตก,Samut Songkhram,Mueang Samut Songkhram,...,Not Change,Normal,[A],SAMUT SONGKHRAM,"RATCHABURI, SAMUT SONGKHRAM",Phetchaburi - Samut Songkhram,"RATCHABURI, SAMUT SONGKHRAM",Central-West,902044,HH


In [7]:
src_df.dtypes

ZONE_TYPE                    object
TEAM_CODE                    object
SCAB_CODE                    object
CCAATT                       object
PROVINCE_KEY                 object
PROVINCE_TH                  object
DISTRICT_TH                  object
SUB_DISTRICT_TH              object
PROVINCE_ENG                 object
DISTRICT_EN                  object
SUB_DISTRICT_EN              object
TDS_RGM                      object
TDS_PROVINCE                 object
TDS_SGMD                     object
ORGID_P                      object
ORG_LEVEL_P                  object
ORGID_G                      object
ORG_LEVEL_G                  object
ORGID_R                      object
ORG_LEVEL_R                  object
ORGID_H                      object
ORG_LEVEL_H                  object
ORGID_AA                     object
ORG_LEVEL_AA                 object
ORG_LEVEL_AA_NAME            object
ORG_LEVEL_AA_DESC            object
ORG_ID_A                     object
ORG_LEVEL_A                 